# 1 Loading Libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# 2 Reading a part of the CSV

In [5]:
skiprows=5113692
nrows=1704564

In [6]:
test=pd.read_csv("../input/testing/test.csv.gzip",compression="gzip",skiprows=range(1,skiprows),nrows=nrows)

# 3 Understanding the Data

# 3.1 Shape of the Data

In [7]:
test.shape

(1704564, 170)

# 3.2 Find the Data Types

In [8]:
test.dtypes

customer_ID     object
S_2             object
P_2            float64
D_39             int64
B_1            float64
                ...   
D_140            int64
D_141          float64
D_143            int64
D_144          float64
D_145            int64
Length: 170, dtype: object

# 3.3 Glimps of Data

In [9]:
test.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,D_41,B_3,D_43,...,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145
0,73595651e1b9f065750a1e2879d2689c98d342a845f625...,2018-12-12,0.837808,15,0.055469,0.400603,0.0,0.0,0.018117,0.102703,...,-1,-1,-1,-1,0,0,0.0,0,0.001559,0
1,73595651e1b9f065750a1e2879d2689c98d342a845f625...,2019-01-06,0.868203,9,0.046672,0.402012,0.0,0.0,0.028462,0.097630,...,-1,-1,-1,-1,0,0,0.0,0,0.005586,0
2,73595651e1b9f065750a1e2879d2689c98d342a845f625...,2019-02-27,0.784160,2,0.052631,0.536164,0.0,0.0,0.032849,0.095478,...,-1,-1,-1,-1,0,0,0.0,0,0.004542,0
3,73595651e1b9f065750a1e2879d2689c98d342a845f625...,2019-03-24,0.755658,0,0.041387,0.534082,0.0,0.0,0.024537,0.097627,...,-1,-1,-1,-1,0,0,0.0,0,0.000458,0
4,73595651e1b9f065750a1e2879d2689c98d342a845f625...,2019-04-14,0.776803,17,0.050129,0.343985,0.0,0.0,0.043422,0.090240,...,-1,-1,-1,-1,0,0,0.0,0,0.004043,0


Thus we can reduce the size of the data to float16

# 4 Data Compression

In [10]:
for col in test.columns:
    if col not in ['customer_ID', 'target', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68','S_2']:
        test[col]=test[col]/(test[col].max()-test[col].min())
    if col not in ['customer_ID', 'target', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68','S_2']:
        test[col]=test[col].astype("float16")

# 5 Label encoding/factorizing the remaining character variables

The categorical features are the followings ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [11]:
columns=['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'] 
for col in columns: 
    print(test[col].value_counts())

 0    1443211
 1     245919
 2      14977
-1        457
Name: B_30, dtype: int64
 2    620966
 3    416589
 1    311950
 5    130593
 7     96008
 4     74068
 6     53933
-1       457
Name: B_38, dtype: int64
 1    1045653
 0     616506
-1      42405
Name: D_114, dtype: int64
 0    1657475
-1      42405
 1       4684
Name: D_116, dtype: int64
 0    468015
 5    366026
 4    342753
 3    185578
 6    147477
 7    120004
-1     42405
 2     32306
Name: D_117, dtype: int64
 0    1319936
 1     342223
-1      42405
Name: D_120, dtype: int64
 2    1291524
 1     353965
 0      46701
-1      12374
Name: D_126, dtype: int64
3    1265136
0     272686
4     138607
1      16808
2       7307
5       4020
Name: D_63, dtype: int64
 0    897874
 3    444055
 2    306046
-1     56589
Name: D_64, dtype: int64
-1    1502482
 1     202082
Name: D_66, dtype: int64
 6    844411
 5    390087
 4    154207
 3    150470
 2     73153
-1     52155
 1     40081
Name: D_68, dtype: int64


* D_64 has category 1 missing which is present in training data set.
* D_66 has category 0 missing which is present in training dataset.
* D_68 has category 0 missing which is present in training dataset.

In [12]:
test.groupby(by="customer_ID")

Here we factorize the date column.

In [13]:
test["S_2"]=test["S_2"].str.slice(start=0,stop=7)
mapping={"2018-04":13,"2018-05":14,"2018-06":15,"2018-07":16,"2018-08":17,"2018-09":18,"2018-10":19,"2018-11":20,"2018-12":21,"2019-01":22,"2019-02":23,"2019-03":24,"2019-04":25,"2019-05":26,"2019-06":27,"2019-07":28,"2019-08":29,"2019-09":30,"2019-10":31} 
test=test.replace({"S_2":mapping}) 
test["S_2"]

0          21
1          22
2          23
3          24
4          25
           ..
1704559    21
1704560    22
1704561    23
1704562    24
1704563    25
Name: S_2, Length: 1704564, dtype: int64

In [14]:
mapping={0:0.183402,1:0.627797,2:0.612930,-1:0.356151}
test=test.replace({"B_30":mapping}) 
test["B_30"]

0          0.183402
1          0.183402
2          0.183402
3          0.183402
4          0.183402
             ...   
1704559    0.627797
1704560    0.627797
1704561    0.627797
1704562    0.627797
1704563    0.183402
Name: B_30, Length: 1704564, dtype: float64

In [15]:
mapping={2:0.063306,3:0.328604,1:0.135264,5:0.559430,4:0.699716,7:0.498865,6:0.614916,-1:0.356151}
test=test.replace({"B_38":mapping}) 
test["B_38"]

0          0.559430
1          0.328604
2          0.328604
3          0.328604
4          0.559430
             ...   
1704559    0.699716
1704560    0.699716
1704561    0.699716
1704562    0.699716
1704563    0.699716
Name: B_38, Length: 1704564, dtype: float64

In [16]:
mapping={1:0.179497,0:0.345989,-1:0.437736}
test=test.replace({"D_114":mapping}) 
test["D_114"]

0          0.345989
1          0.345989
2          0.345989
3          0.345989
4          0.345989
             ...   
1704559    0.345989
1704560    0.345989
1704561    0.345989
1704562    0.345989
1704563    0.345989
Name: D_114, Length: 1704564, dtype: float64

In [17]:
mapping={0:0.242413,-1:0.437736,1:0.613341}
test=test.replace({"D_116":mapping}) 
test["D_116"]

0          0.242413
1          0.242413
2          0.242413
3          0.242413
4          0.242413
             ...   
1704559    0.242413
1704560    0.242413
1704561    0.242413
1704562    0.242413
1704563    0.242413
Name: D_116, Length: 1704564, dtype: float64

In [18]:
mapping={0:0.273270,4:0.269573,5:0.173700,3:0.340783,6:0.135087,7:0.134076,-1:0.437736,2:0.446632}
test=test.replace({"D_117":mapping}) 
test["D_117"]

0          0.173700
1          0.173700
2          0.173700
3          0.173700
4          0.173700
             ...   
1704559    0.269573
1704560    0.269573
1704561    0.269573
1704562    0.269573
1704563    0.269573
Name: D_117, Length: 1704564, dtype: float64

In [19]:
mapping={0:0.212713,1:0.471097,-1:0.437736}
test=test.replace({"D_120":mapping}) 
test["D_120"]

0          0.212713
1          0.212713
2          0.212713
3          0.212713
4          0.212713
             ...   
1704559    0.471097
1704560    0.471097
1704561    0.471097
1704562    0.471097
1704563    0.471097
Name: D_120, Length: 1704564, dtype: float64

In [20]:
mapping={2:0.232809,1:0.309875,0:0.209538,-1:0.468044}
test=test.replace({"D_126":mapping}) 
test["D_126"]

0          0.232809
1          0.232809
2          0.232809
3          0.232809
4          0.232809
             ...   
1704559    0.232809
1704560    0.232809
1704561    0.232809
1704562    0.232809
1704563    0.232809
Name: D_126, Length: 1704564, dtype: float64

In [21]:
mapping={3:0.262817,0:0.169865,4:0.290894,1:0.190918,2:0.237401,5:0.317444}
test=test.replace({"D_63":mapping}) 
test["D_63"]

0          0.262817
1          0.262817
2          0.262817
3          0.262817
4          0.262817
             ...   
1704559    0.262817
1704560    0.262817
1704561    0.262817
1704562    0.262817
1704563    0.262817
Name: D_63, Length: 1704564, dtype: float64

In [22]:
mapping={0:0.169255,3:0.343223,2:0.315242,-1:0.408642,1:0.220723}
test=test.replace({"D_64":mapping}) 
test["D_64"]

0          0.315242
1          0.315242
2          0.315242
3          0.315242
4          0.315242
             ...   
1704559    0.343223
1704560    0.343223
1704561    0.343223
1704562    0.343223
1704563    0.343223
Name: D_64, Length: 1704564, dtype: float64

In [23]:
mapping={-1:0.257171,1:0.183504,0:0.383747}
test=test.replace({"D_66":mapping}) 
test["D_66"]

0          0.257171
1          0.257171
2          0.257171
3          0.257171
4          0.257171
             ...   
1704559    0.257171
1704560    0.257171
1704561    0.257171
1704562    0.257171
1704563    0.257171
Name: D_66, Length: 1704564, dtype: float64

In [24]:
mapping={6:0.174952,5:0.262219,3:0.369780,4:0.343603,2:0.392902,-1:0.414493,1:0.403990,0:0.179717}
test=test.replace({"D_68":mapping}) 
test["D_68"]

0          0.174952
1          0.174952
2          0.174952
3          0.174952
4          0.174952
             ...   
1704559    0.262219
1704560    0.262219
1704561    0.262219
1704562    0.262219
1704563    0.262219
Name: D_68, Length: 1704564, dtype: float64

In [25]:
x=pd.DataFrame(np.linspace(start=12,stop=0,num=13))
y=pd.DataFrame([0.258934,0.257893,0.257176,0.256362,0.254698,0.252840,0.250988,0.248830,0.245882,0.242539,0.236346,0.238807,0.232017])

mapping={}
for i in range(13,32):
    mapping[i]=(x[0].corr(y[0])*y[0].std()/x[0].std())*(i-x[0].mean())+y[0].mean()
test=test.replace({"S_2":mapping}) 
test["S_2"]

0          0.281961
1          0.284177
2          0.286393
3          0.288609
4          0.290826
             ...   
1704559    0.281961
1704560    0.284177
1704561    0.286393
1704562    0.288609
1704563    0.290826
Name: S_2, Length: 1704564, dtype: float64

# 6 Normalise all the numerical columns

Here I normalize the numerical columns and change the data type to save some space. Here we have used min max normalizer.

There are still some null values which was giving us trouble during prediction. So we will remove them here.

In [26]:
for col in test.columns:
    if col not in ['customer_ID', 'target', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68','S_2']:
        test[col].fillna(value=test[col].mean(),inplace=True)
        test[col]=test[col]/(test[col].max()-test[col].min())
    else:
        test[col].fillna(value=test[col].mode(),inplace=True)
    if col not in ['customer_ID', 'target']:
        test[col]=test[col].astype("float16")

In [27]:
test.isnull().values.sum()

0

# 7 Upload the test data

In [28]:
test.to_csv("test_1.csv.gzip",compression="gzip")